## FASE 1. Extracción de datos de películas

In [1]:
# importamos todas las librerias necesarias para realizar el ejercicio

import requests
import pandas as pd
import mysql.connector
from mysql.connector import Error
import numpy as np

In [18]:
# extraemos los datos de la API y la guardamos en una variable (en formato JSON)

url = "https://beta.adalab.es/resources/apis/pelis/pelis.json"

datos = requests.get(url)
datos.status_code
datos.content

datos_pelis = datos.json()

## FASE 2. Creación de la base de datos

In [ ]:
# Iniciamos la connexion con SQL ---> Se deberá modificar el password acorde con el del usuario que ejecute el codigo. 
try:
    cnx = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='',   #he eliminado el campo password al subir el repo
    )
    print('Conexión exitosa')
except Error as e:
    print('Error al conectar:', e)


Conexión exitosa


In [4]:
# Creamos la base de datos 

try:
    mycursor = cnx.cursor()
    query = "CREATE DATABASE IF NOT EXISTS films_ejercicio_2"
    mycursor.execute(query)
    print("Query exitosa")
except:
    print("Error.")

Query exitosa


In [5]:
# para crear las tablas primero debemos revisar los datos del API y analizar que columnas debemos crear.
datos_pelis

[{'id': 1,
  'titulo': 'The Godfather',
  'año': 1972,
  'duracion': 175,
  'genero': 'Crimen',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 2,
  'titulo': 'The Godfather Part II',
  'año': 1974,
  'duracion': 202,
  'genero': 'Crimen',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 3,
  'titulo': 'Pulp Fiction',
  'año': 1994,
  'duracion': 154,
  'genero': 'Crimen',
  'adultos': True,
  'subtitulos': ['es', 'en']},
 {'id': 4,
  'titulo': 'Forrest Gump',
  'año': 1994,
  'duracion': 142,
  'genero': 'Drama',
  'adultos': False,
  'subtitulos': ['es', 'en', 'fr']},
 {'id': 5,
  'titulo': 'The Dark Knight',
  'año': 2008,
  'duracion': 152,
  'genero': 'Acción',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 6,
  'titulo': 'Fight Club',
  'año': 1999,
  'duracion': 139,
  'genero': 'Drama',
  'adultos': True,
  'subtitulos': ['es', 'en']},
 {'id': 7,
  'titulo': 'Inception',
  'año': 2010,
  'duracion': 148,
  'genero': 'Ciencia ficción',
  'adultos

In [6]:
# Creamos la tabla 

mycursor.execute("USE films_ejercicio_2")

mycursor.execute("DROP TABLE IF EXISTS films") #añado el drop table if exists por si tengo que volver a ejecutar el codigo con alguna modificacion

query = '''CREATE TABLE films (
    id INT PRIMARY KEY AUTO_INCREMENT,
    titulo VARCHAR(255) NOT NULL,
    año INT,
    duracion FLOAT,
    genero VARCHAR(255),
    adultos TINYINT(1),
    subtitulos TEXT
);'''

mycursor.execute(query)

## Fase 3. Inserción de los Datos en la Base de datos

In [7]:
#creamos un DF
df_films = pd.DataFrame(datos_pelis)


# Reemplazar NaN con None para que se inserten como NULL en la BD
df_films_clean = df_films.replace({np.nan: None, 'nan': None, 'NaN': None})


In [8]:
# Convertir DataFrame directamente a lista de tuplas

columnas = ['titulo','año','duracion','genero','adultos','subtitulos'] 
datos_films_clean = df_films_clean[columnas].values.tolist()  #selecciono las columnas de las que quiero hacer el insert, así no incorporo el ID



In [9]:
datos_films_clean

[['The Godfather', 1972, 175, 'Crimen', False, ['es', 'en']],
 ['The Godfather Part II', 1974, 202, 'Crimen', False, ['es', 'en']],
 ['Pulp Fiction', 1994, 154, 'Crimen', True, ['es', 'en']],
 ['Forrest Gump', 1994, 142, 'Drama', False, ['es', 'en', 'fr']],
 ['The Dark Knight', 2008, 152, 'Acción', False, ['es', 'en']],
 ['Fight Club', 1999, 139, 'Drama', True, ['es', 'en']],
 ['Inception', 2010, 148, 'Ciencia ficción', False, ['es', 'en', 'de']],
 ['The Matrix', 1999, 136, 'Ciencia ficción', False, ['es', 'en']],
 ['The Shawshank Redemption', 1994, 142, 'Drama', False, ['es', 'en']],
 ['Interstellar', 2014, 169, 'Ciencia ficción', False, ['es', 'en']],
 ['Gladiator', 2000, 155, 'Acción', False, ['es', 'en']],
 ['Titanic', 1997, 195, 'Romance', False, ['es', 'en', 'fr']],
 ['Avatar', 2009, 162, 'Ciencia ficción', False, ['es', 'en']],
 ['Saving Private Ryan', 1998, 169, 'Bélico', True, ['es', 'en']],
 ['The Silence of the Lambs', 1991, 118, 'Thriller', True, ['es', 'en']],
 ['Se7en', 1

In [10]:
# BOLEANOS. En SQL se representan con el TINYINT que es 1 o 0 , por lo que la columna de ADULTOS debemos cambiarla para que nos devuelva 0 o 1.
df_films_clean['adultos'] = df_films_clean['adultos'].apply(lambda x: int(x))

In [11]:
# para poder insertar los datos a SQL, debe estar en formato lista. 
# En este momento, los datos contenidos en "subtitulos" nos darian error dado que es una lista dentro de una lista.
# Una opción es pasar el contenido a String
df_films_clean['subtitulos'] = df_films_clean['subtitulos'].apply(
    lambda x: ",".join(x) if isinstance(x, list) else x )

In [ ]:
#comprobamos que adultos nos devuelve 0 o 1 y que los subtitulos ya no es una lista 
df_films_clean  

,id,titulo,año,duracion,genero,adultos,subtitulos
0,1,The Godfather,1972,175,Crimen,0,"es,en"
1,2,The Godfather Part II,1974,202,Crimen,0,"es,en"
2,3,Pulp Fiction,1994,154,Crimen,1,"es,en"
3,4,Forrest Gump,1994,142,Drama,0,"es,en,fr"
4,5,The Dark Knight,2008,152,Acción,0,"es,en"
...,...,...,...,...,...,...,...
95,96,La vita è bella,1997,116,Drama,0,"es,en,it"
96,97,Requiem for a Dream,2000,102,Drama,1,"es,en"
97,98,Memento,2000,113,Thriller,1,"es,en"
98,99,Eternal Sunshine of the Spotless Mind,2004,108,Drama,0,"es,en"


In [ ]:
#ahora sí, podemos pasar los datos del dataFrame a lista para poder insertarlo a la tabla
datos_films = df_films_clean[columnas].values.tolist()

In [15]:
# Query de inserción
query_insert = """
INSERT INTO films (titulo, año, duracion, genero, adultos, subtitulos)
                              
VALUES (%s, %s, %s, %s, %s, %s)
"""

In [16]:
# Ejecutar inserción
mycursor.executemany(query_insert, datos_films)
cnx.commit()

print(f"{mycursor.rowcount} registros insertados")

print(f"{mycursor.rowcount} registros insertados")

100 registros insertados
100 registros insertados


In [ ]:
#Una vez terminamos el flujo de trabajo de inserción. Si dejamos de trabajar usando la connexion con SQL. Podriamos cerrarla.
mycursor.close()
cnx.close()
print('Conexión cerrada')

Conexión cerrada


NOTA_Posibles mejoras:
- Crear tabla de idiomas para los subtitulos?
- Crear tabla de genero (son datos que se repiten)

## FASE 4. Obtener información a partir de los datos.



In [ ]:
# Si hemos cerrado la connexion con SQL annteriormente. Ahora debemos iniciarla de nuevo ---> Se deberá modificar el password acorde con el del usuario que ejecute el codigo. 
# Dado que hemos creado el Database en el apartado anterior. podemos ya abrir 
try:
    cnx = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='',    #he eliminado el campo password al subir el repo
        database = 'films_ejercicio_2'
    )
    print('Conexión exitosa')
except Error as e:
    print('Error al conectar:', e)


Conexión exitosa


In [50]:
mycursor = cnx.cursor()
mycursor.execute("USE films_ejercicio_2") # no haría falta si en el punto anterior hemos realizado la connexión ya usando este database

¿Cuántas películas tienen una duración superior a 120 minutos?

In [53]:
query = """
SELECT COUNT(*)
FROM films
WHERE duracion > 120
"""
mycursor.execute(query)
results_duracion = mycursor.fetchall()
results_duracion

[(59,)]

¿Cuántas películas incluyen subtítulos en español?

In [55]:
query = """
SELECT COUNT(*)
FROM films
WHERE subtitulos LIKE "%ES%";
"""
mycursor.execute(query)
results_duracion = mycursor.fetchall()
results_duracion

[(100,)]

¿Cuántas películas tienen contenido adulto?

In [56]:
query = """
SELECT COUNT(*)
FROM films
WHERE adultos = 1;
"""
mycursor.execute(query)
results_duracion = mycursor.fetchall()
results_duracion

[(47,)]

¿Cuál es la película más antigua registrada en la base de datos?

In [66]:
query = """
SELECT titulo, año
FROM films
ORDER BY año
LIMIT 1
"""
mycursor.execute(query)
results_duracion = mycursor.fetchall()
results_duracion

[('Citizen Kane', 1941)]

Muestra el promedio de duración de las películas agrupado por género.

In [67]:
query = """
SELECT ROUND(AVG(duracion),2) AS promedio_duracion, genero
FROM films
GROUP BY genero;
"""

#En este caso muestro el resultado en un data frame asi vemos las columnas y sus cabeceras
df = pd.read_sql(query, cnx)
df


C:\Users\laura\AppData\Local\Temp\ipykernel_31080\1018756946.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)


,promedio_duracion,genero
0,154.29,Crimen
1,126.26,Drama
2,139.44,Acción
3,136.31,Ciencia ficción
4,139.67,Romance
5,161.00,Bélico
6,121.67,Thriller
7,128.00,Musical
8,159.80,Fantasía
9,133.00,Aventura


¿Cuántas películas por año se han registrado en la base de datos? Ordena de mayor a menor.

In [68]:
query = """
SELECT count(id) AS total_peliculas, año
FROM films
GROUP BY año
ORDER BY total_peliculas DESC
"""


df = pd.read_sql(query, cnx)
df


C:\Users\laura\AppData\Local\Temp\ipykernel_31080\2953748035.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)


,total_peliculas,año
0,5,2001
1,4,2013
2,4,1994
3,4,2008
4,4,1999
5,4,2017
6,3,2010
7,3,1998
8,3,2014
9,3,2000


In [71]:
query = """
SELECT año
FROM films
GROUP BY año
ORDER BY COUNT(*) DESC
LIMIT 1;
"""


mycursor.execute(query)
results_duracion = mycursor.fetchall()
results_duracion

[(2001,)]

Obtén un listado de todos los géneros y cuántas películas corresponden a cada uno.

In [72]:
query = """
SELECT genero, count(id)
FROM films
GROUP BY genero
"""


df = pd.read_sql(query, cnx)
df

C:\Users\laura\AppData\Local\Temp\ipykernel_31080\469406158.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)


,genero,count(id)
0,Crimen,7
1,Drama,27
2,Acción,9
3,Ciencia ficción,13
4,Romance,3
5,Bélico,2
6,Thriller,6
7,Musical,1
8,Fantasía,5
9,Aventura,3


Muestra todas las películas cuyo título contenga la palabra "Godfather" (puedes usar cualquier palabra).

In [73]:
query = """
SELECT titulo
FROM films
WHERE titulo LIKE '%GODFATHER%';
"""


mycursor.execute(query)
results_duracion = mycursor.fetchall()
results_duracion

[('The Godfather',), ('The Godfather Part II',)]